# Simulation of Audio-Visual Duration Estimation with Causal Inference Model
This notebook simulates the audio-visual duration estimation task using a causal inference model. It allows you to explore how different parameters affect the estimates through interactive controls.
### Experimenter knows the true durations:
Standard duration($S_s$) is always 0.5s, test duration varies($S_t$).
We simulate and give the parameters to the model, and see how the estimates change.
Parameters:
- $\sigma_a$: Standard deviation of auditory measurement noise
- $\sigma_v$: Standard deviation of visual measurement noise
- $p_c$: Prior probability of common cause
### Parameters we dont need to adjust specifically:
- $c$: Conflict level between auditory and visual stimuli
- $tmin$ and $tmax$: Minimum and maximum test durations

### Steps:
1. Create and repeated array of standard durations: S_s: [0.5, 0.5, ..., 0.5]
2. Create an array of duration differences: delta: [0.0, 0.1, ..., 2.0]
3. Create constant conflict durations: c:[-250,-167,-83,0,83,167,250]ms
4. For each combination of S_s, delta, and c:
    - Calculate test duration: S_t = S_s + delta + c
5. For simplicity we dont need to estimate noisy measurements, we can directly use the true durations as measurements.
6. Use the causal inference model to compute the final estimates based on the measurements and parameters.


In [1]:
# import lbraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm, gaussian_kde
from ipywidgets import interact, FloatSlider, widgets
from scipy.special import expit  # Sigmoid function for probability mapping
import warnings


In [2]:
## Simulation of Audio-Visual Duration Estimation with Causal Inference Model This notebook simulates the audio-visual duration estimation task using a causal inference model. It allows you to explore how different parameters affect the estimates through interactive controls. ### Experimenter knows the true durations: Standard duration($S_s$) is always 0.5s, test duration varies($S_t$). We simulate and give the parameters to the model, and see how the estimates change. Parameters: - $\sigma_a$: Standard deviation of auditory measurement noise - $\sigma_v$: Standard deviation of visual measurement noise - $p_c$: Prior probability of common cause ### Parameters we dont need to adjust specifically: - $c$: Conflict level between auditory and visual stimuli - $\tmin$ and $\tmax$: Minimum and maximum test durations ### Steps: 1. Create and repeated array of standard durations: S_s: [0.5, 0.5, ..., 0.5] 2. Create an array of duration differences: delta: [0.0, 0.1, ..., 2.0] 3. Create constant conflict durations: c:[-250,-167,-83,0,83,167,250]ms 4. For each combination of S_s, delta, and c: - Calculate test duration: S_t = S_s + delta + c 5. For simplicity we dont need to estimate noisy measurements, we can directly use the true durations as measurements. 6. Use the causal inference model to compute the final estimates based on the measurements and parameters.
# Standard durations of auditory stimuli
S_a_s = 0.5  # seconds
# repeat
S_a_s = np.repeat(S_a_s, 9990)
# Range of test durations (difference from standard) -100% to +100%
delta_percent = np.linspace(-0.95, 0.95, 7)  # -100% to +100%
# Convert percentage differences to absolute time differences (in seconds)
delta = delta_percent * S_a_s[0]  # since S_a_s is constant
# Constant conflict levels (in seconds)
c = np.array([-0.45,-0.35, -0.25, -0.167, -0.083, 0.0, 0.083, 0.167, 0.25, 0.35,+0.45])  # seconds

# Ensure all arrays have the same length
min_length = min(len(S_a_s), len(delta) * (len(S_a_s) // len(delta)), len(c) * (len(S_a_s) // len(c)))
S_a_s = S_a_s[:min_length]
delta = np.tile(delta, int(np.ceil(len(S_a_s) / len(delta))))[:min_length]
c = np.tile(c, int(np.ceil(len(S_a_s) / len(c))))[:min_length]

# Calculate test durations
S_a_t = S_a_s + delta 
S_v_s = S_a_s + c  # Visual test durations with conflict
S_v_t = S_a_t # no conflict in test durations for visual
# Combine standard and test durations, delta, and conflict into a single dataset
simData=np.column_stack((S_a_s, S_a_t, S_v_s, S_v_t, delta, c))
# Columns: [S_a_s, S_a_t, S_v_s, S_v_t, delta, c]

# create a dataframe
import pandas as pd
simData = pd.DataFrame(simData, columns=['S_a_s', 'S_a_t', 'S_v_s', 'S_v_t', 'delta', 'c'])





In [3]:
simData

S_a_s     S_a_t  S_v_s     S_v_t     delta      c
0       0.5  0.025000  0.050  0.025000 -0.475000 -0.450
1       0.5  0.183333  0.150  0.183333 -0.316667 -0.350
2       0.5  0.341667  0.250  0.341667 -0.158333 -0.250
3       0.5  0.500000  0.333  0.500000  0.000000 -0.167
4       0.5  0.658333  0.417  0.658333  0.158333 -0.083
...     ...       ...    ...       ...       ...    ...
9983    0.5  0.183333  0.583  0.183333 -0.316667  0.083
9984    0.5  0.341667  0.667  0.341667 -0.158333  0.167
9985    0.5  0.500000  0.750  0.500000  0.000000  0.250
9986    0.5  0.658333  0.850  0.658333  0.158333  0.350
9987    0.5  0.816667  0.950  0.816667  0.316667  0.450

[9988 rows x 6 columns]

In [4]:
# Check for potential issues with negative durations
print("Data summary:")
print(f"S_a_t min: {simData['S_a_t'].min():.3f}, max: {simData['S_a_t'].max():.3f}")
print(f"S_v_t min: {simData['S_v_t'].min():.3f}, max: {simData['S_v_t'].max():.3f}")
print(f"S_v_s min: {simData['S_v_s'].min():.3f}, max: {simData['S_v_s'].max():.3f}")

# Check if any values are negative or zero
negative_Sa_t = simData[simData['S_a_t'] <= 0]
negative_Sv_t = simData[simData['S_v_t'] <= 0] 
negative_Sv_s = simData[simData['S_v_s'] <= 0]

print(f"\nNegative/zero durations:")
print(f"S_a_t <= 0: {len(negative_Sa_t)} cases")
print(f"S_v_t <= 0: {len(negative_Sv_t)} cases") 
print(f"S_v_s <= 0: {len(negative_Sv_s)} cases")

if len(negative_Sv_s) > 0:
    print(f"Example negative S_v_s: {negative_Sv_s[['S_v_s', 'c']].head()}")

Data summary:
S_a_t min: 0.025, max: 0.975
S_v_t min: 0.025, max: 0.975
S_v_s min: 0.050, max: 0.950

Negative/zero durations:
S_a_t <= 0: 0 cases
S_v_t <= 0: 0 cases
S_v_s <= 0: 0 cases


In [14]:
# --- FUSION MODEL IMPLEMENTATION ---
def fusion_estimate(m_a, m_v, sigma_a, sigma_v):
    """Bayesian optimal integration of auditory and visual measurements."""
    # Calculate weights based on reliabilities (inverse variances)
    w_a = 1 / sigma_a**2
    w_v = 1 / sigma_v**2
    # Compute the fused estimate
    fused_estimate = (w_a * m_a + w_v * m_v) / (w_a + w_v)
    return fused_estimate

# --- CAUSAL INFERENCE MODEL IMPLEMENTATION ---
def p_single(m,sigma,t_min,t_max):
    """p(m | C=2) and Gaussian measurement noise N(m; y, sigma^2)."""
    hi_cdf= norm.cdf((t_max - m) /sigma)
    lo_cdf=norm.cdf((t_min-m)/sigma)
    return (hi_cdf-lo_cdf)/(t_max-t_min)

# Causal inference model for duration estimation
def L_C1(m_a,m_v,sigma_a,sigma_v,t_min,t_max):
    sigma_c_sq = (sigma_a**2 * sigma_v**2) / (sigma_a**2 + sigma_v**2)
    sigma_c = np.sqrt(sigma_c_sq)
    mu_c = (m_a / sigma_a**2 + m_v / sigma_v**2) / (1 / sigma_a**2 + 1 / sigma_v**2)

    hi_cdf = norm.cdf((t_max-mu_c)/sigma_c)
    lo_cdf = norm.cdf((t_min-mu_c)/sigma_c)
    
    expo = np.exp(-(m_a-m_v)**2/(2*(sigma_a**2+sigma_v**2)))
    
    prior = 1/(t_max-t_min)
    return prior * sigma_c/np.sqrt(sigma_a**2 * sigma_v**2) * (hi_cdf-lo_cdf) * expo

def L_C2(m_a,m_v,sigma_a,sigma_v,t_min,t_max):
    """ Likelihood of separate sources: product of two marginal likelihoods 
        two integral over two hidden duration y_a y_v"""
    return p_single(m_a,sigma_a,t_min,t_max) * p_single(m_v,sigma_v,t_min,t_max)

def posterior_C1(m_a,m_v,sigma_a,sigma_v,p_c,t_min,t_max):
    """ Posterior probability of common cause P(C=1 | m_a,m_v) """
    # Likelihoods under each causal structure
    L1 = L_C1(m_a,m_v,sigma_a,sigma_v,t_min,t_max)
    L2 = L_C2(m_a,m_v,sigma_a,sigma_v,t_min,t_max)
    # Unnormalized posteriors
    post_C1_unnorm = L1 * p_c
    post_C2_unnorm = L2 * (1 - p_c)
    # Normalization constant
    norm_const = post_C1_unnorm + post_C2_unnorm
    
    # Handle numerical stability
    if np.isscalar(norm_const):
        if norm_const == 0:
            return 0.5
    else:
        norm_const = np.where(norm_const == 0, 1e-10, norm_const)
    
    # Posterior probabilities
    post_C1 = post_C1_unnorm / norm_const
    return post_C1

def causal_inference_estimate(m_a, m_v, sigma_a, sigma_v, p_c, t_min, t_max, model):
    """Causal inference duration estimate (stays in current space - log or linear)"""
    # Posterior probability of common cause
    p_C1 = posterior_C1(m_a, m_v, sigma_a, sigma_v, p_c, t_min, t_max)
    
    # Estimate under common cause (fused estimate)
    est_C1 = fusion_estimate(m_a, m_v, sigma_a, sigma_v)
    # Estimate under separate causes (auditory estimate)
    est_C2 = m_a
    
    # Final estimate as a weighted average
    final_estimate = p_C1 * est_C1 + (1 - p_C1) * est_C2
    
    if model == "log-space":
        final_estimate = p_C1 * est_C1 + (1 - p_C1) * est_C2

    return final_estimate

def forced_fusion_estimate(m_a, m_v, sigma_a, sigma_v, model):
    """Forced fusion model - always fuses, no causal inference"""
    fused = fusion_estimate(m_a, m_v, sigma_a, sigma_v)
    
    if model == "log-space":
        # Convert back to linear space
        return np.exp(fused)
    else:
        return fused

def estimate_duration(m_a,m_v,sigma_a,sigma_v,p_c,t_min,t_max,lambda_=0.1, model="linear-space"):
    """ Final duration estimate as a weighted average of estimates under each causal structure """
    
    if model == "log-space":
        # For log-space model: measurements are in log space, causal inference in log space
        # t_min and t_max should also be in log space
        log_t_min = np.log(t_min)
        log_t_max = np.log(t_max)        
        # Do causal inference in log space
        final_estimate = causal_inference_estimate(m_a, m_v, sigma_a, sigma_v, p_c, log_t_min, log_t_max, model=model)
    else:  # linear-space
        # For linear-space model: everything in linear space
        final_estimate = causal_inference_estimate(m_a, m_v, sigma_a, sigma_v, p_c, t_min, t_max, model=model)
    
    return final_estimate

def estimate_probability_matching(m_a, m_v, sigma_a, sigma_v, p_c, t_min, t_max, model="linear-space"):
    """ Probability matching model - sample from the posterior distribution """
    if model == "log-space":
        t_min = np.log(t_min)
        t_max = np.log(t_max)
    # Calculate posterior probability of common cause
    post_C1 = posterior_C1(m_a, m_v, sigma_a, sigma_v, p_c, t_min, t_max)
    est_fused = fusion_estimate(m_a, m_v, sigma_a, sigma_v)
    est_separate = m_a
    # bernoulli sampling based on posterior
    from scipy.stats import bernoulli
    b = bernoulli(post_C1).rvs()
    
    # Probability matching estimate based on sampled causal structure
    final_estimate = b * est_fused + (1 - b) * est_separate
    
    if model == "log-space":
        final_estimate = np.exp(final_estimate)
    
    return final_estimate

    

# Calculate probabilities of choosing test longer than standard for each condition
def prob_test_longer_condition(S_a_s, S_a_t, S_v_s, S_v_t, sigma_a, sigma_v, p_c, t_min, t_max, lambda_=0.1, measurement="linear-space",model="lognorm", nSimul=1000, use_forced_fusion=False):
    """ Calculate the probability of choosing the test duration as longer than the standard duration for a specific condition. """
    
    # Check for invalid durations (≤ 0) when using log-space
    if measurement == "log-space":
        min_duration = 1e-6  # Very small positive number
        S_a_s = max(S_a_s, min_duration)
        S_a_t = max(S_a_t, min_duration)
        S_v_s = max(S_v_s, min_duration)
        S_v_t = max(S_v_t, min_duration)
    
    # Generate measurements with noise
    if measurement == "log-space":
        # Measurements are Gaussian in log space (like your main implementation)
        m_a_s = np.random.normal(np.log(S_a_s), scale=sigma_a, size=nSimul)
        m_a_t = np.random.normal(np.log(S_a_t), scale=sigma_a, size=nSimul)
        m_v_s = np.random.normal(np.log(S_v_s), scale=sigma_v, size=nSimul)
        m_v_t = np.random.normal(np.log(S_v_t), scale=sigma_v, size=nSimul)
    elif measurement == "linear-space":
        m_a_s = np.random.normal(S_a_s, scale=sigma_a, size=nSimul)
        m_a_t = np.random.normal(S_a_t, scale=sigma_a, size=nSimul)
        m_v_s = np.random.normal(S_v_s, scale=sigma_v, size=nSimul)
        m_v_t = np.random.normal(S_v_t, scale=sigma_v, size=nSimul)


    
    if model=="ForcedFusion":
        # Forced fusion: always integrate, no causal inference
        est_standard = forced_fusion_estimate(m_a_s, m_v_s, sigma_a, sigma_v, model)
        est_test = forced_fusion_estimate(m_a_t, m_v_t, sigma_a, sigma_v, model)
    elif model=="CausalInference":
        # Causal inference model
        est_standard = estimate_duration(m_a_s, m_v_s, sigma_a, sigma_v, p_c, t_min, t_max, lambda_, model)
        est_test = estimate_duration(m_a_t, m_v_t, sigma_a, sigma_v, p_c, t_min, t_max, lambda_, model)
    elif model=="ProbablityMatching":
        # Probability matching model
        est_standard = estimate_probability_matching(m_a_s, m_v_s, sigma_a, sigma_v, p_c, t_min, t_max, model)
        est_test = estimate_probability_matching(m_a_t, m_v_t, sigma_a, sigma_v, p_c, t_min, t_max, model)
    else:
        raise ValueError("Invalid model specified. Choose 'ForcedFusion', 'CausalInference', or 'ProbablityMatching'.")
    # Calculate probability
    p_base = np.mean(est_test > est_standard)
    p_final = (1 - lambda_) * p_base + lambda_ * 0.5
    return p_final


# Updated simulation function that generates psychometric curves
def simulate_duration_estimation(sigma_a=0.05, sigma_v=0.1, p_c=0.5, lambda_=0.1,measurement="linear-space", model="lognorm", nSimul=1000, t_min=0.01, t_max=3.0):
    """
    Simulate duration estimation and generate psychometric curves for different conflict levels.
    """
    # Get unique conditions
    unique_deltas = np.sort(simData['delta'].unique())
    unique_conflicts = np.sort(simData['c'].unique())
    
    # Initialize results storage
    results = []
    results_fusion = []  # For forced fusion model
    
    print(f"Simulating with {len(unique_deltas)} deltas and {len(unique_conflicts)} conflict levels...")
    print(f"Model: {model}")
    
    # For each conflict level
    for conflict in unique_conflicts:
        conflict_results = []
        conflict_results_fusion = []
        
        # For each delta (test duration difference)
        for delta in unique_deltas:
            # Get the stimulus durations for this condition
            S_a_s = 0.5  # Standard auditory duration
            S_a_t = S_a_s + delta  # Test auditory duration
            S_v_s = S_a_s + conflict  # Standard visual duration (with conflict)
            S_v_t = S_a_t  # Test visual duration (same as auditory test)
            
            # Skip conditions with non-positive durations for log-space
            if model == "log-space" and (S_a_t <= 0 or S_v_t <= 0):
                print(f"Skipping condition: delta={delta:.3f}, conflict={conflict:.3f} (negative duration)")
                continue
            
            # Calculate probability for causal inference model
            p_longer = prob_test_longer_condition(
                S_a_s, S_a_t, S_v_s, S_v_t, 
                sigma_a, sigma_v, p_c, t_min, t_max, lambda_, measurement, model, nSimul, use_forced_fusion=False
            )
            
            # Calculate probability for forced fusion model
            p_longer_fusion = prob_test_longer_condition(
                S_a_s, S_a_t, S_v_s, S_v_t, 
                sigma_a, sigma_v, p_c, t_min, t_max, lambda_, measurement, model, nSimul, use_forced_fusion=True
            )
            
            conflict_results.append({
                'delta': delta,
                'delta_ms': delta * 1000,
                'conflict': conflict,
                'conflict_ms': conflict * 1000,
                'p_test_longer': p_longer,
                'S_a_s': S_a_s,
                'S_a_t': S_a_t,
                'S_v_s': S_v_s,
                'S_v_t': S_v_t
            })
            
            conflict_results_fusion.append({
                'delta': delta,
                'delta_ms': delta * 1000,
                'conflict': conflict,
                'conflict_ms': conflict * 1000,
                'p_test_longer': p_longer_fusion,
                'S_a_s': S_a_s,
                'S_a_t': S_a_t,
                'S_v_s': S_v_s,
                'S_v_t': S_v_t
            })
        
        results.extend(conflict_results)
        results_fusion.extend(conflict_results_fusion)
    
    # Convert to DataFrame for easier plotting
    results_df = pd.DataFrame(results)
    results_fusion_df = pd.DataFrame(results_fusion)
    
    if len(results_df) == 0:
        print("No valid conditions found!")
        return results_df
    
    # Plotting
    plt.figure(figsize=(16, 8))
    
    # Plot 1: Psychometric curves for each conflict level
    plt.subplot(1, 2, 1)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_conflicts)))
    
    # Plot causal inference model (solid lines)
    for i, conflict in enumerate(unique_conflicts):
        conflict_data = results_df[results_df['conflict'] == conflict]
        if len(conflict_data) > 0:
            plt.plot(conflict_data['delta_ms'], conflict_data['p_test_longer'], 
                    'o-', color=colors[i], label=f'c: {conflict*1000:.0f}ms', alpha=0.8, linewidth=2)
    
    # Plot forced fusion model (dashed lines)
    for i, conflict in enumerate(unique_conflicts):
        conflict_data = results_fusion_df[results_fusion_df['conflict'] == conflict]
        if len(conflict_data) > 0:
            plt.plot(conflict_data['delta_ms'], conflict_data['p_test_longer'], 
                    '-', color=colors[i], alpha=0.6, linewidth=0.5)
    
    plt.title('Psychometric Curves: Causal Inference vs Forced Fusion')
    plt.xlabel('Test Duration Difference from Standard (ms)')
    plt.ylabel('P(Choosing Test as Longer)')
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.axhline(0.5, color='gray', linestyle='--', alpha=0.5)
    plt.axvline(0, color='gray', linestyle='--', alpha=0.5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    
    # Plot 2: PSE comparison for both models
    plt.subplot(1, 2, 2)
    pse_by_conflict = []
    pse_by_conflict_fusion = []
    
    # Calculate PSE for causal inference model
    for conflict in unique_conflicts:
        conflict_data = results_df[results_df['conflict'] == conflict].sort_values('delta_ms')
        if len(conflict_data) > 1:
            try:
                p_values = conflict_data['p_test_longer'].values
                delta_values = conflict_data['delta_ms'].values
                
                if p_values.min() <= 0.5 <= p_values.max():
                    pse = np.interp(0.5, p_values, delta_values)
                else:
                    closest_idx = np.argmin(np.abs(p_values - 0.5))
                    pse = delta_values[closest_idx]
                
                pse_by_conflict.append({
                    'conflict_ms': conflict*1000, 
                    'pse_ms': pse,
                    'conflict': conflict
                })
            except:
                pse_by_conflict.append({
                    'conflict_ms': conflict*1000, 
                    'pse_ms': 0,
                    'conflict': conflict
                })
    
    # Calculate PSE for forced fusion model
    for conflict in unique_conflicts:
        conflict_data = results_fusion_df[results_fusion_df['conflict'] == conflict].sort_values('delta_ms')
        if len(conflict_data) > 1:
            try:
                p_values = conflict_data['p_test_longer'].values
                delta_values = conflict_data['delta_ms'].values
                
                if p_values.min() <= 0.5 <= p_values.max():
                    pse = np.interp(0.5, p_values, delta_values)
                else:
                    closest_idx = np.argmin(np.abs(p_values - 0.5))
                    pse = delta_values[closest_idx]
                
                pse_by_conflict_fusion.append({
                    'conflict_ms': conflict*1000, 
                    'pse_ms': pse,
                    'conflict': conflict
                })
            except:
                pse_by_conflict_fusion.append({
                    'conflict_ms': conflict*1000, 
                    'pse_ms': 0,
                    'conflict': conflict
                })
    
    # Plot both PSE curves
    if pse_by_conflict:
        pse_df = pd.DataFrame(pse_by_conflict)
        plt.plot(pse_df['conflict_ms'], pse_df['pse_ms'], 'ro-', linewidth=2, markersize=8, label='Causal Inference')
        
    if pse_by_conflict_fusion:
        pse_fusion_df = pd.DataFrame(pse_by_conflict_fusion)
        plt.plot(pse_fusion_df['conflict_ms'], pse_fusion_df['pse_ms'], 'bs--', linewidth=2, markersize=8, label='Forced Fusion', alpha=0.7)
    
    plt.title('PSE vs Conflict: Causal Inference vs Forced Fusion')
    plt.xlabel('Conflict Level (ms)')
    plt.ylabel('PSE (ms from standard)')
    plt.ylim(-500, 500)
    plt.grid(True, alpha=0.3)
    plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
    plt.legend(loc='best')
    
    # Vertical cut lines at ±250 ms
    ax = plt.gca()
    ax.axvline(+250, color='black', linestyle='-', alpha=0.3, label='±250ms bounds')
    ax.axvline(-250, color='black', linestyle='-', alpha=0.3)

    # Shade regions beyond the ±250 ms bounds
    xmin, xmax = ax.get_xlim()

    # Shade to the right of +250 ms
    if xmax > 250:
        ax.axvspan(max(250, xmin), xmax, color='lightgray', alpha=0.2, linewidth=0)

    # Shade to the left of -250 ms
    if xmin < -250:
        ax.axvspan(xmin, min(-250, xmax), color='lightgray', alpha=0.2, linewidth=0)
    
    plt.tight_layout()
    plt.show()

# Run the simulation with log-space model (your main model)
# print("Running causal inference simulation...")
# results = simulate_duration_estimation(sigma_a=0.05, sigma_v=0.1, p_c=0.5, lambda_=0.1, model="log-space", nSimul=1000)

In [ ]:
# Interactive widget for parameter exploration
def interactive_simulation(sigma_a=0.05, sigma_v=0.1, p_c=0.5, lambda_=0.1, model_space="linear-space", n_simulations=500, t_min=0.01, t_max=3.0, modelName="CausalInference"):
    """Interactive version of the simulation with parameter controls"""
    results = simulate_duration_estimation(
        sigma_a=sigma_a, 
        sigma_v=sigma_v, 
        p_c=p_c, 
        lambda_=lambda_, 
        measurement=model_space, 
        nSimul=n_simulations,
        t_min=t_min,
        t_max=t_max,
        model=modelName
    )
    return results

# Create interactive widget
from ipywidgets import interact, FloatSlider, Dropdown, IntSlider

interact(
    interactive_simulation,
    sigma_a=FloatSlider(value=0.5, min=0.01, max=1, step=0.01, description='σₐ (auditory):'),
    sigma_v=FloatSlider(value=0.3, min=0.01, max=0.8, step=0.01, description='σᵥ (visual):'),
    p_c=FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='p_c (common cause):'),
    lambda_=FloatSlider(value=0.1, min=0.0, max=0.5, step=0.01, description='λ (lapse rate):'),
    model_space=Dropdown(options=['linear-space', 'log-space'], value='log-space', description='Measurement space:'),
    n_simulations=IntSlider(value=1000, min=100, max=2000, step=100, description='N simulations:'),
    t_min=FloatSlider(value=0.01, min=0.001, max=0.5, step=0.001, description='t_min (s):'),
    t_max=FloatSlider(value=3.0, min=0.5, max=5.0, step=0.01, description='t_max (s):'),
    modelName=Dropdown(options=['CausalInference', 'ForcedFusion', 'ProbablityMatching'], value='CausalInference', description='Model:')
);

interactive(children=(FloatSlider(value=0.5, description='σₐ (auditory):', max=1.0, min=0.01, step=0.01), Floa…